In [3]:
import urllib.request as req
import numpy as np
import pandas as pd
import time
import math
import bs4
from fake_useragent import UserAgent

In [11]:
def getRoot(url):
    
    ua = UserAgent()
    
    data = try_request(url)
        
    root = bs4.BeautifulSoup(data, "html.parser")
    return root

def try_request(url):
    try:
        ua = UserAgent()
        request = req.Request(url, headers={"User-Agent" : ua.random})
        
        with req.urlopen(request) as response:
            data = response.read().decode("utf-8")
    except:
        print("error, wait for 10 sec")
        time.sleep(10)
        data = try_request(url)
    return data

def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False
    
def getGameData(summaryUrl):
    # summaryUrl = game.find_all("td")[13].a.get("href").replace("..", "https://gol.gg")
    summaryRoot = getRoot(summaryUrl);

    allStatsUrl = summaryUrl.replace("page-game","page-fullstats")
    allStatsRoot = getRoot(allStatsUrl)

    # blue team win/lose
    title = summaryRoot.find('title').text.replace(" - Games of Legends", "").split(" - ")[0]
    tag = summaryRoot.find('title').text.replace(" - Games of Legends", "").split(" - ")[1].split(" ")[-1]
    tournament = summaryRoot.find('title').text.replace(" - Games of Legends", "").split(" - ")[1].replace(" "+tag, "")
    date = summaryRoot.find('div', class_='col-12 col-sm-5 text-right').text.split(' (')[0]
    gametime = summaryRoot.find('div', class_='col-6 text-center').h1.text
    blue_team = summaryRoot.find_all("div", class_="col-12 blue-line-header")[0].a.text
    red_team = summaryRoot.find_all("div", class_="col-12 red-line-header")[0].a.text
    label = "red" if "LOSS" in summaryRoot.find_all("div", class_="col-12 blue-line-header")[0].text else "blue"

    blue_info = []
    red_info = []

    # tower, dragon, baron
    for i in range(1,4):
        if len(summaryRoot.find_all("span", class_="score-box blue_line")) < 5:
            valid = False
            break
        blue_info.append(float(summaryRoot.find_all("span", class_="score-box blue_line")[i].text))
        red_info.append(float(summaryRoot.find_all("span", class_="score-box red_line")[i].text))

    # kill, death, assist, CSM, GPM, G%, VSPM, WPM, VMPM, WCPM, VS%, DPM, DMG%, GD@15, CSD@15, XPD@15, DT
    indices = [4, 5, 6, 11, 13, 14, 20, 21, 22, 23, 24, 29, 30, 38, 39, 40, 51]
    trs = allStatsRoot.table.find_all("tr")
    valid = True
    for index in indices:
        if len(trs) <= index:
            valid = False
            continue
        tds = trs[index].find_all("td")
        for i in range(1,11):
            if len(tds) <= i:
                valid = False
                continue
            value = tds[i].text.replace("%","")
            if isfloat(value) and i <= 5:
                blue_info.append(float(value))
            elif isfloat(value) and i >=6:
                red_info.append(float(value))
            else:
                valid = False
    
    return valid, [title, tournament, tag, date, blue_team, red_team, label, gametime] + blue_info + red_info

def try_getGameData(url):
    try:
        valid, data = getGameData(summaryUrl)
    except:
        print("error, wait for 100 sec")
        time.sleep(100)
        valid, data = try_getGameData(url)
    return valid, data

In [12]:
tournament_names = pd.read_table('./data/tournaments.txt',header=None)[0].tolist()

# web crawler 
dataset = []
invalid_cnt = 0
for tournament_name in tournament_names:
    matchlistUrl = "https://gol.gg/tournament/tournament-matchlist/"+tournament_name.replace(" ", "%20")+"/"
    matchlistRoot = getRoot(matchlistUrl)
    trs = matchlistRoot.find('tbody').find_all('tr')
    for tr in trs:
        serieUrl = tr.a.get('href').replace("..", "https://gol.gg")
        if "page-game" in serieUrl:
            summaryUrls = [serieUrl]
        else:
            summaryUrls = []
            serieRoot = getRoot(serieUrl)
            pageTitle = serieRoot.find('h1').text
            links = serieRoot.find_all('a', 'nav-link')
            for link in links:
                if pageTitle == link.get('title'): summaryUrls.append(link.get('href').replace("..", "https://gol.gg"))
        for summaryUrl in summaryUrls:
            
            valid, data = try_getGameData(summaryUrl)
            
            if valid: dataset.append(data) 
            else: invalid_cnt += 1
            
            print(data[0] + " - " + data[1], data[2])
            if valid: print("Loading, valid: ", len(dataset), "invalid: ", invalid_cnt)
            else: print("Load failed, valid:", len(dataset), "invalid:", invalid_cnt)
            
            # if len(dataset)%100 == 0: pd.DataFrame(dataset).to_csv('./data/lol.csv', index = False)

CES vs Team Secret game 1 - VCS Winter 2021 WEEK3
Loading, valid:  1 invalid:  0
CES vs Team Secret game 2 - VCS Winter 2021 WEEK3
Loading, valid:  2 invalid:  0
SE vs SGB game 1 - VCS Winter 2021 WEEK3
Loading, valid:  3 invalid:  0
SE vs SGB game 2 - VCS Winter 2021 WEEK3
Loading, valid:  4 invalid:  0
SE vs SGB game 3 - VCS Winter 2021 WEEK3
Loading, valid:  5 invalid:  0
Team Flash vs LX game 1 - VCS Winter 2021 WEEK3
Loading, valid:  6 invalid:  0
Team Flash vs LX game 2 - VCS Winter 2021 WEEK3
Loading, valid:  7 invalid:  0
Team Flash vs LX game 3 - VCS Winter 2021 WEEK3
Loading, valid:  8 invalid:  0
SKY vs GAM Esports game 1 - VCS Winter 2021 WEEK3
Loading, valid:  9 invalid:  0
SKY vs GAM Esports game 2 - VCS Winter 2021 WEEK3
Loading, valid:  10 invalid:  0
SE vs CES game 1 - VCS Winter 2021 WEEK3
Loading, valid:  11 invalid:  0
SE vs CES game 2 - VCS Winter 2021 WEEK3
Loading, valid:  12 invalid:  0
Team Flash vs Team Secret game 1 - VCS Winter 2021 WEEK3
Loading, valid:  13

ANO vs UDG game 1 - GLL Winter 2021 WEEK4
Loading, valid:  106 invalid:  0
WLG vs Team Refuse game 1 - GLL Winter 2021 WEEK4
Loading, valid:  107 invalid:  0
TP vs GOAL game 1 - GLL Winter 2021 WEEK4
Loading, valid:  108 invalid:  0
GSMC vs ZRL game 1 - GLL Winter 2021 WEEK4
Loading, valid:  109 invalid:  0
TP vs ANO game 1 - GLL Winter 2021 WEEK4
Loading, valid:  110 invalid:  0
WLG vs GSMC game 1 - GLL Winter 2021 WEEK4
Loading, valid:  111 invalid:  0
GOAL vs ZRL game 1 - GLL Winter 2021 WEEK4
Loading, valid:  112 invalid:  0
Team Refuse vs UDG game 1 - GLL Winter 2021 WEEK4
Loading, valid:  113 invalid:  0
WLG vs ANO game 1 - GLL Winter 2021 WEEK3
Loading, valid:  114 invalid:  0
GOAL vs GSMC game 1 - GLL Winter 2021 WEEK3
Loading, valid:  115 invalid:  0
Team Refuse vs TP game 1 - GLL Winter 2021 WEEK3
Loading, valid:  116 invalid:  0
UDG vs ZRL game 1 - GLL Winter 2021 WEEK3
Loading, valid:  117 invalid:  0
GSMC vs ANO game 1 - GLL Winter 2021 WEEK3
Loading, valid:  118 invalid: 

FTW vs OFF7 game 1 - LPLOL Grand Final 2021 ROUND2
Loading, valid:  195 invalid:  5
FTW vs OFF7 game 2 - LPLOL Grand Final 2021 ROUND2
Loading, valid:  196 invalid:  5
FTW vs OFF7 game 3 - LPLOL Grand Final 2021 ROUND2
Loading, valid:  197 invalid:  5
FTW vs OFF7 game 4 - LPLOL Grand Final 2021 ROUND2
Loading, valid:  198 invalid:  5
KRM vs WD game 1 - LPLOL Grand Final 2021 ROUND2
Loading, valid:  199 invalid:  5
KRM vs WD game 2 - LPLOL Grand Final 2021 ROUND2
Loading, valid:  200 invalid:  5
KRM vs WD game 3 - LPLOL Grand Final 2021 ROUND2
Loading, valid:  201 invalid:  5
KRM vs WD game 4 - LPLOL Grand Final 2021 ROUND2
Loading, valid:  202 invalid:  5
KRM vs WD game 5 - LPLOL Grand Final 2021 ROUND2
Loading, valid:  203 invalid:  5
OFF7 vs GTZ Bulls game 1 - LPLOL Grand Final 2021 ROUND1
Loading, valid:  204 invalid:  5
OFF7 vs GTZ Bulls game 2 - LPLOL Grand Final 2021 ROUND1
Loading, valid:  205 invalid:  5
OFF7 vs GTZ Bulls game 3 - LPLOL Grand Final 2021 ROUND1
Loading, valid:  

Team Liquid vs MAD Lions game 1 - World Championship 2021 DAY1
Loading, valid:  287 invalid:  5
100 Thieves vs EDG game 1 - World Championship 2021 DAY1
Loading, valid:  288 invalid:  5
T1 vs DFM game 1 - World Championship 2021 DAY1
Loading, valid:  289 invalid:  5
GEN vs LNG Esports game 1 - World Championship 2021 DAY1
Loading, valid:  290 invalid:  5
Fnatic vs HLE game 1 - World Championship 2021 DAY1
Loading, valid:  291 invalid:  5
RNG vs PSG Talon game 1 - World Championship 2021 DAY1
Loading, valid:  292 invalid:  5
FPX vs DWG KIA game 1 - World Championship 2021 DAY1
Loading, valid:  293 invalid:  5
Cloud9 vs PEACE game 1 - World Championship Play-In 2021 QUAL.ROUND
Loading, valid:  294 invalid:  5
Cloud9 vs PEACE game 2 - World Championship Play-In 2021 QUAL.ROUND
Loading, valid:  295 invalid:  5
Cloud9 vs PEACE game 3 - World Championship Play-In 2021 QUAL.ROUND
Loading, valid:  296 invalid:  5
HLE vs BYG game 1 - World Championship Play-In 2021 QUAL.ROUND
Loading, valid:  2

100A vs WC game 2 - LCS Proving Grounds Summer 2021 ROUND4
Loading, valid:  376 invalid:  5
100A vs WC game 3 - LCS Proving Grounds Summer 2021 ROUND4
Loading, valid:  377 invalid:  5
TLA vs DIGA game 1 - LCS Proving Grounds Summer 2021 ROUND4
Loading, valid:  378 invalid:  5
TLA vs DIGA game 2 - LCS Proving Grounds Summer 2021 ROUND4
Loading, valid:  379 invalid:  5
TLA vs DIGA game 3 - LCS Proving Grounds Summer 2021 ROUND4
Loading, valid:  380 invalid:  5
C9 vs C9A game 1 - LCS Proving Grounds Summer 2021 ROUND2
Loading, valid:  381 invalid:  5
C9 vs C9A game 2 - LCS Proving Grounds Summer 2021 ROUND2
Loading, valid:  382 invalid:  5
FLYA vs GGA game 1 - LCS Proving Grounds Summer 2021 ROUND2
Loading, valid:  383 invalid:  5
FLYA vs GGA game 2 - LCS Proving Grounds Summer 2021 ROUND2
Loading, valid:  384 invalid:  5
FLYA vs GGA game 3 - LCS Proving Grounds Summer 2021 ROUND2
Loading, valid:  385 invalid:  5
EGA vs IMTA game 1 - LCS Proving Grounds Summer 2021 ROUND2
Loading, valid: 

...

Team WE vs Vici Gaming game 2 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8848
LGD Gaming vs Snake game 1 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8849
LGD Gaming vs Snake game 2 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8850
Energy Pacemaker vs Gamtee game 1 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8851
Energy Pacemaker vs Gamtee game 2 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8852
Edward Gaming vs IG game 1 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8853
Edward Gaming vs IG game 2 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8854
Team WE vs LGD game 1 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8855
Team WE vs LGD game 2 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8856
SHRC vs King game 1 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8857
SHRC vs King game 2 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8858
OMG vs Snake game 1 - LPLSpring2015 
Load failed, valid: 20846 invalid: 8859
OMG vs

CJ Entus vs KT Rolster game 1 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8945
CJ Entus vs KT Rolster game 2 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8946
CJ Entus vs KT Rolster game 3 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8947
IM vs Samsung game 1 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8948
IM vs Samsung game 2 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8949
IM vs Samsung game 3 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8950
JinAir vs GE Tigers game 1 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8951
JinAir vs GE Tigers game 2 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8952
SKT T1 vs KT Rolster game 1 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8953
SKT T1 vs KT Rolster game 2 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8954
SKT T1 vs KT Rolster game 3 - ChampionsSpring2015 
Load failed, valid: 20846 invalid: 8955
Najin vs CJ Entu

TieBreaker - C9 vs CLG game 1
Load failed, valid: 20846 invalid: 9037
TieBreaker - T8 vs TL game 1
Load failed, valid: 20846 invalid: 9038
CLG vs Team Impulse game 1 - NA LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9039
Gravity vs Team 8 game 1 - NA LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9040
Dignitas vs Team Liquid game 1 - NA LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9041
Cloud 9 vs TSM game 1 - NA LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9042
Winterfox vs Team Coast game 1 - NA LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9043
Team Impulse vs Dignitas game 1 - NA LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9044
Team Coast vs TSM game 1 - NA LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9045
Winterfox vs Team Liquid game 1 - NA LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9046
Team 8 vs CLG game 1 - NA LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9047
Gra

Team Impulse vs Team Liquid game 1 - NA LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9125
Winterfox vs Gravity game 1 - NA LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9126
Team Coast vs Dignitas game 1 - NA LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9127
TSM vs Cloud 9 game 1 - NA LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9128
TieBreaker - MYM vs Giants game 1
Load failed, valid: 20846 invalid: 9129
H2K Gaming vs UoL game 1 - EU LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9130
MYM vs Roccat game 1 - EU LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9131
Gambit vs SK Gaming game 1 - EU LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9132
Fnatic vs Elements game 1 - EU LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9133
CW vs Giants game 1 - EU LCS Spring 2015 Week 9
Load failed, valid: 20846 invalid: 9134
UoL vs Gambit game 1 - EU LCS Spring 2015 Week 9
Load failed, valid: 20846 inval

Giants Gaming vs CW game 1 - EU LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9213
MYM vs SK Gaming game 1 - EU LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9214
Gambit vs Unicons of Love game 1 - EU LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9215
SK Gaming vs Roccat game 1 - EU LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9216
CW vs H2K Gaming game 1 - EU LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9217
Giants vs MYM game 1 - EU LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9218
Elements vs Fnatic game 1 - EU LCS Spring 2015 Week 1
Load failed, valid: 20846 invalid: 9219
TSM vs World Elite game 1 - IEM Katowice 2015 FINALS
Load failed, valid: 20846 invalid: 9220
TSM vs World Elite game 2 - IEM Katowice 2015 FINALS
Load failed, valid: 20846 invalid: 9221
TSM vs World Elite game 3 - IEM Katowice 2015 FINALS
Load failed, valid: 20846 invalid: 9222
GE Tigers vs World Elite game 1 - IEMKatowice2015 
Load failed,

In [13]:
# pd.DataFrame(dataset).to_csv('./data/lol.csv', index = False)